In [21]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [22]:
df_control = pd.read_excel("ab_testing.xlsx", sheet_name="Control Group")
df_test = pd.read_excel("ab_testing.xlsx", sheet_name="Test Group")

Verileri analiz edelim.

In [23]:
df_control.head()

,Impression,Click,Purchase,Earning
0,82529.45927,6090.07732,665.21125,2311.27714
1,98050.45193,3382.86179,315.08489,1742.80686
2,82696.02355,4167.96575,458.08374,1797.82745
3,109914.40040,4910.88224,487.09077,1696.22918
4,108457.76263,5987.65581,441.03405,1543.72018


In [24]:
df_control.dtypes

Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object

In [25]:
df_control.shape

(40, 4)

In [26]:
df_control.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,101711.44907,20302.15786,45475.94296,85726.69035,99790.70108,115212.81654,147539.33633
Click,40.00000,5100.65737,1329.98550,2189.75316,4124.30413,5001.22060,5923.80360,7959.12507
Purchase,40.00000,550.89406,134.10820,267.02894,470.09553,531.20631,637.95709,801.79502
Earning,40.00000,1908.56830,302.91778,1253.98952,1685.84720,1975.16052,2119.80278,2497.29522


In [27]:
df_control.isnull().sum()

Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

In [28]:
df_test.head()

,Impression,Click,Purchase,Earning
0,120103.50380,3216.54796,702.16035,1939.61124
1,134775.94336,3635.08242,834.05429,2929.40582
2,107806.62079,3057.14356,422.93426,2526.24488
3,116445.27553,4650.47391,429.03353,2281.42857
4,145082.51684,5201.38772,749.86044,2781.69752


In [29]:
df_test.shape

(40, 4)

In [30]:
df_test.dtypes

Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object

In [31]:
df_test.isnull().sum()

Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

In [32]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,120512.41176,18807.44871,79033.83492,112691.97077,119291.30077,132050.57893,158605.92048
Click,40.00000,3967.54976,923.09507,1836.62986,3376.81902,3931.35980,4660.49791,6019.69508
Purchase,40.00000,582.10610,161.15251,311.62952,444.62683,551.35573,699.86236,889.91046
Earning,40.00000,2514.89073,282.73085,1939.61124,2280.53743,2544.66611,2761.54540,3171.48971


Verileri birleştirelim.

In [33]:
df_control["Group"] = "Control"
df_test["Group"] = "Test"

In [34]:
df = pd.concat([df_control, df_test], axis = 0, ignore_index=False)

In [35]:
df

,Impression,Click,Purchase,Earning,Group
0,82529.45927,6090.07732,665.21125,2311.27714,Control
1,98050.45193,3382.86179,315.08489,1742.80686,Control
2,82696.02355,4167.96575,458.08374,1797.82745,Control
3,109914.40040,4910.88224,487.09077,1696.22918,Control
4,108457.76263,5987.65581,441.03405,1543.72018,Control
...,...,...,...,...,...
35,79234.91193,6002.21358,382.04712,2277.86398,Test
36,130702.23941,3626.32007,449.82459,2530.84133,Test
37,116481.87337,4702.78247,472.45373,2597.91763,Test
38,79033.83492,4495.42818,425.35910,2595.85788,Test


**AB Testi için hipotezi tanımlayalım.**

- H0 : M1 = M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında fark yoktur.)
- H1 : M1!= M2 (... vardır.)

In [49]:
df.groupby("Group").agg({"Purchase":"mean"})

,Purchase
Group,
Control,550.89406
Test,582.10610


In [42]:
# 2. Varsayım Kontrolü
    # 1. Normallik Varsayımı
        # H0: Normallik varsayımı sağlanmaktadır.
        # H1: Normallik varsayımı sağlanmamaktadır.

In [44]:
test_stat, p_value = shapiro(df.loc[df["Group"] == "Control", "Purchase"])
print(f"Test Stats:{test_stat} p_value:{p_value}")

Test Stats:0.9772694110870361 p_value:0.5891125202178955


In [45]:
test_stat, p_value = shapiro(df.loc[df["Group"] == "Test", "Purchase"])
print(f"Test Stats:{test_stat} p_value:{p_value}")

Test Stats:0.9589453935623169 p_value:0.15413342416286469


p_value değerleri 0.05'ten büyük olduğu için normallik varsayımı sağlanmaktadır.

In [46]:
# 2. Varyans Homojenliği Varsayımı
    # H0: Varyanslar Homojendir.
    # H1: Varyanslar Homojen Değildir.

test_stat, p_value = levene(df.loc[df["Group"] == "Control", "Purchase"], df.loc[df["Group"] == "Test", "Purchase"])
print(f"Test Stats:{test_stat} p_value:{p_value}")

Test Stats:2.6392694728747363 p_value:0.10828588271874791


p_value değeri 0.05'ten büyük olduğu için Varyanslar homojendir. İki varsayım da sağlandığı için **bağımsız iki örneklem t testi** yapılmalıdır.

In [47]:
test_stat, p_value = ttest_ind(df.loc[df["Group"] == "Control", "Purchase"], df.loc[df["Group"] == "Test", "Purchase"], equal_var=True)
print(f"Test Stats:{test_stat} p_value:{p_value}")

Test Stats:-0.9415584300312966 p_value:0.34932579202108416


p_value değeri 0.05'ten büyük olduğu için H0 Hipotezi **reddedilemez**. Kontrol grubu ve test grubu satın alma ortalamaları arasında fark yoktur.

### Sonuçların Analiz Edilmesi

Yeni geçilen sistemin satış ortalaması yönünden eski sistemle bir farkı yoktur. Bu yüzden yeni sisteme geçmek hem insan hem de finansal yönden bir maliyet oluşturacaksa pek mantıklı değildir. Mevcut olan sistemi kullanmak daha mantıklı olur. Yeni sistem daha gelişmiş bir hale geldiği zaman tekrardan analiz yapılabilir.